# 주식가격 하락으로 마음아픈 나를 위로해주는 챗봇 구현하기
 - 참조: https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html

In [48]:
!nvidia-smi

Fri May 26 11:34:40 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 527.56       Driver Version: 527.56       CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0  On |                  N/A |
|  0%   45C    P8    15W / 170W |   1592MiB / 12288MiB |     10%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

##### 0) 라이브러리 불러오기

In [49]:
# (1) 기본 라이브러리
import random
import pandas as pd
from tqdm import tqdm

# (2) 딥러닝 라이브러리
import torch
import torch.nn as nn
from torch import optim

In [50]:
torch.manual_seed(0)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [51]:
device

device(type='cuda')

##### 1) 데이터 불러오기
 - 단어 표현을 위해 단어사전 만들기 단어 --> index, index --> 단어

In [52]:
#!pip install Korpora
#from Korpora import Korpora
#Korpora.corpus_list()

In [53]:
!git clone https://github.com/songys/Chatbot_data.git

fatal: destination path 'Chatbot_data' already exists and is not an empty directory.


In [54]:
raw_dataset = pd.read_csv("./Chatbot_data/ChatbotData.csv")

In [55]:
raw_dataset

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0
...,...,...,...
11818,훔쳐보는 것도 눈치 보임.,티가 나니까 눈치가 보이는 거죠!,2
11819,훔쳐보는 것도 눈치 보임.,훔쳐보는 거 티나나봐요.,2
11820,흑기사 해주는 짝남.,설렜겠어요.,2
11821,힘든 연애 좋은 연애라는게 무슨 차이일까?,잘 헤어질 수 있는 사이 여부인 거 같아요.,2


In [56]:
qa_list = raw_dataset[["Q", "A"]] # !!!
qa_list = qa_list.values.tolist() # !!!

In [57]:
qa_list

[['12시 땡!', '하루가 또 가네요.'],
 ['1지망 학교 떨어졌어', '위로해 드립니다.'],
 ['3박4일 놀러가고 싶다', '여행은 언제나 좋죠.'],
 ['3박4일 정도 놀러가고 싶다', '여행은 언제나 좋죠.'],
 ['PPL 심하네', '눈살이 찌푸려지죠.'],
 ['SD카드 망가졌어', '다시 새로 사는 게 마음 편해요.'],
 ['SD카드 안돼', '다시 새로 사는 게 마음 편해요.'],
 ['SNS 맞팔 왜 안하지ㅠㅠ', '잘 모르고 있을 수도 있어요.'],
 ['SNS 시간낭비인 거 아는데 매일 하는 중', '시간을 정하고 해보세요.'],
 ['SNS 시간낭비인데 자꾸 보게됨', '시간을 정하고 해보세요.'],
 ['SNS보면 나만 빼고 다 행복해보여', '자랑하는 자리니까요.'],
 ['가끔 궁금해', '그 사람도 그럴 거예요.'],
 ['가끔 뭐하는지 궁금해', '그 사람도 그럴 거예요.'],
 ['가끔은 혼자인게 좋다', '혼자를 즐기세요.'],
 ['가난한 자의 설움', '돈은 다시 들어올 거예요.'],
 ['가만 있어도 땀난다', '땀을 식혀주세요.'],
 ['가상화폐 쫄딱 망함', '어서 잊고 새출발 하세요.'],
 ['가스불 켜고 나갔어', '빨리 집에 돌아가서 끄고 나오세요.'],
 ['가스불 켜놓고 나온거 같아', '빨리 집에 돌아가서 끄고 나오세요.'],
 ['가스비 너무 많이 나왔다.', '다음 달에는 더 절약해봐요.'],
 ['가스비 비싼데 감기 걸리겠어', '따뜻하게 사세요!'],
 ['가스비 장난 아님', '다음 달에는 더 절약해봐요.'],
 ['가장 확실한 건 뭘까?', '가장 확실한 시간은 오늘이에요. 어제와 내일을 놓고 고민하느라 시간을 낭비하지 마세요.'],
 ['가족 여행 가기로 했어', '온 가족이 모두 마음에 드는 곳으로 가보세요.'],
 ['가족 여행 고고', '온 가족이 모두 마음에 드는 곳으로 가보세요.'],
 ['가족 여행 어디로 가지?', '온 가족이 모두 마음에 드는 곳으로 가보세요.']

##### 2) 단어 사전 생성

In [58]:
# 참조 모두의 딥러닝 시즌2
SOS_token = 0 # 문장의 시작
EOS_token = 1 # 문장의 마지막
UNK_token = 2 # 미지 단어

# 단어 사전 생성을 위한 Class
class Vocab:
    def __init__(self):
        self.vocab2index = {"<SOS>": SOS_token, "<EOS>": EOS_token, "<UNK>": UNK_token}
        self.index2vocab = {SOS_token: "<SOS>", EOS_token: "<EOS>", UNK_token: "<UNK>"}
        self.vocab_count = {} # !!
        self.n_vocab = len(self.vocab2index)

    def add_vocab(self, sentence):
        for word in sentence.split(" "):
            if word not in self.vocab2index:
                self.vocab2index[word] = self.n_vocab
                self.vocab_count[word] = 1 # !!
                self.index2vocab[self.n_vocab] = word
                self.n_vocab += 1
            else:
                self.vocab_count[word] += 1


In [59]:
vocab = Vocab()
vocab.add_vocab("안녕 나는 경태라고해")

In [60]:
vocab.vocab2index["안녕"]

3

##### 3) Source & Target 단어장 분할 생성

In [61]:
def preprocess_df(corpus, source_max_length, target_max_length): # 병렬 연산을 위해 source max length보다 작은 길이의 문장이 들어오면 남은 길이 공간에 pad를 끼워넣음
    print("reading corpus...")
    pairs = corpus

    source_vocab = Vocab()
    target_vocab = Vocab()

    print("Counting words...")
    for pair in pairs:
        source_vocab.add_vocab(pair[0]) # !
        target_vocab.add_vocab(pair[1]) # !
    print("source vocab size =", source_vocab.n_vocab)
    print("target vocab size =", target_vocab.n_vocab)

    return pairs, source_vocab, target_vocab

In [62]:
SOURCE_MAX_LENGTH = 50
TARGET_MAX_LENGTH = 50
load_pairs, load_source_vocab, load_target_vocab = preprocess_df(qa_list, source_max_length=SOURCE_MAX_LENGTH, target_max_length=TARGET_MAX_LENGTH) # !!!

reading corpus...
Counting words...
source vocab size = 14287
target vocab size = 10008


In [63]:
load_pairs[:3]

[['12시 땡!', '하루가 또 가네요.'],
 ['1지망 학교 떨어졌어', '위로해 드립니다.'],
 ['3박4일 놀러가고 싶다', '여행은 언제나 좋죠.']]

In [64]:
load_source_vocab.vocab2index["위로해"]

12742

In [65]:
load_target_vocab.vocab2index["위로해"]

6

In [66]:
load_source_vocab.vocab2index["안녕"], load_source_vocab.index2vocab[2]

(4006, '<UNK>')

In [67]:
load_source_vocab.n_vocab, load_target_vocab.n_vocab

(14287, 10008)

##### 4) 문장 String -> Index Tensor

In [68]:
def str_to_indexes_tensor(vocab, sentence):
    indexes = [vocab.vocab2index[word] if word in vocab.vocab2index else UNK_token for word in sentence.split(" ")]
    indexes.append(vocab.vocab2index["<EOS>"]) # !!
    return torch.Tensor(indexes).long().to(device).view(-1, 1) # [indexes_size] -> [indexes_size, 1] # !!!

In [69]:
a_sample_tensor = str_to_indexes_tensor(load_source_vocab, "학원폭력 짜증나")

In [70]:
a_sample_tensor

tensor([[6098],
        [ 918],
        [   1]], device='cuda:0')

In [71]:
a_sample_tensor.size()

torch.Size([3, 1])

##### 5-1) 모델 함수 정의

(1) Encoder 함수 정의

In [72]:
source_emb = nn.Embedding(load_source_vocab.n_vocab, 100)
source_emb.to(device) # !!

source_emb # [Vocab_size, hidden_size]

Embedding(14287, 100)

In [73]:
print(a_sample_tensor.size()) # torch.Size([3, 1])

a_sample_tensor_emb = source_emb(a_sample_tensor.to(device)) # !!! 11-3 코드에서는 가장 앞의 텐서 size가 1이었는데, 이번에는 가장 뒤의 텐서 size가 1이네

print(a_sample_tensor_emb.size()) # torch.Size([3, 1, 100]) # !!
print(a_sample_tensor_emb)

torch.Size([3, 1])
torch.Size([3, 1, 100])
tensor([[[ 0.1127, -0.1983, -0.4962,  1.2140, -2.1170,  0.9750,  0.6317,
           0.5635,  1.7521,  1.0841, -0.3285,  0.5405,  0.7763, -0.4149,
           0.2281,  2.0070,  0.4139,  0.5892, -0.8072,  1.4428, -0.1432,
          -0.4790, -1.9436,  1.8163, -0.0508,  0.8145,  0.2852,  0.9804,
          -0.1242,  1.4256, -0.4671,  0.4610, -0.9637, -0.0788, -0.0571,
          -0.5886, -1.9806,  0.0398,  0.8602, -2.0859, -0.8153, -0.3319,
           0.2828, -1.2567, -0.4173, -1.4092, -0.3097, -0.8915,  0.2312,
          -0.4062, -1.0440, -2.0088,  0.3214,  0.3455, -1.5352,  3.0451,
           1.4223, -0.1017,  1.2486, -0.2031, -2.1639,  0.3630,  0.8738,
           0.3272,  0.3794, -0.0127,  0.3511, -0.9578, -0.4538, -1.0328,
           1.0395,  0.5937, -0.0794, -0.5214, -0.0589, -0.7881, -0.7386,
           0.4456,  0.7102, -0.7494, -1.3087, -2.4207,  0.3503,  1.4739,
           0.1325,  1.2581,  0.5878, -0.3866,  0.5322,  3.6285,  0.6040,
        

In [74]:
a_sample_tensor_emb = a_sample_tensor_emb.squeeze()

print(a_sample_tensor_emb.size()) # torch.Size([3, 100])
print(a_sample_tensor_emb)

torch.Size([3, 100])
tensor([[ 0.1127, -0.1983, -0.4962,  1.2140, -2.1170,  0.9750,  0.6317,  0.5635,
          1.7521,  1.0841, -0.3285,  0.5405,  0.7763, -0.4149,  0.2281,  2.0070,
          0.4139,  0.5892, -0.8072,  1.4428, -0.1432, -0.4790, -1.9436,  1.8163,
         -0.0508,  0.8145,  0.2852,  0.9804, -0.1242,  1.4256, -0.4671,  0.4610,
         -0.9637, -0.0788, -0.0571, -0.5886, -1.9806,  0.0398,  0.8602, -2.0859,
         -0.8153, -0.3319,  0.2828, -1.2567, -0.4173, -1.4092, -0.3097, -0.8915,
          0.2312, -0.4062, -1.0440, -2.0088,  0.3214,  0.3455, -1.5352,  3.0451,
          1.4223, -0.1017,  1.2486, -0.2031, -2.1639,  0.3630,  0.8738,  0.3272,
          0.3794, -0.0127,  0.3511, -0.9578, -0.4538, -1.0328,  1.0395,  0.5937,
         -0.0794, -0.5214, -0.0589, -0.7881, -0.7386,  0.4456,  0.7102, -0.7494,
         -1.3087, -2.4207,  0.3503,  1.4739,  0.1325,  1.2581,  0.5878, -0.3866,
          0.5322,  3.6285,  0.6040, -0.7675, -1.2893, -0.7238, -0.0949, -0.1193,
       

In [75]:
a_sample_tensor_emb.view(1, 1, -1)

tensor([[[ 0.1127, -0.1983, -0.4962,  1.2140, -2.1170,  0.9750,  0.6317,
           0.5635,  1.7521,  1.0841, -0.3285,  0.5405,  0.7763, -0.4149,
           0.2281,  2.0070,  0.4139,  0.5892, -0.8072,  1.4428, -0.1432,
          -0.4790, -1.9436,  1.8163, -0.0508,  0.8145,  0.2852,  0.9804,
          -0.1242,  1.4256, -0.4671,  0.4610, -0.9637, -0.0788, -0.0571,
          -0.5886, -1.9806,  0.0398,  0.8602, -2.0859, -0.8153, -0.3319,
           0.2828, -1.2567, -0.4173, -1.4092, -0.3097, -0.8915,  0.2312,
          -0.4062, -1.0440, -2.0088,  0.3214,  0.3455, -1.5352,  3.0451,
           1.4223, -0.1017,  1.2486, -0.2031, -2.1639,  0.3630,  0.8738,
           0.3272,  0.3794, -0.0127,  0.3511, -0.9578, -0.4538, -1.0328,
           1.0395,  0.5937, -0.0794, -0.5214, -0.0589, -0.7881, -0.7386,
           0.4456,  0.7102, -0.7494, -1.3087, -2.4207,  0.3503,  1.4739,
           0.1325,  1.2581,  0.5878, -0.3866,  0.5322,  3.6285,  0.6040,
          -0.7675, -1.2893, -0.7238, -0.0949, -0.11

In [76]:
rnn = torch.nn.RNN(100, 40, batch_first = True) # [Batch, Sequence, Input_Size]

In [77]:
# declare simple encoder
class Encoder(nn.Module):
    # 1] 모델 생성
    def __init__(self, input_size, hidden_size):
        super(Encoder, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size) # nn.Embedding(input_size = Vocab 사이즈, hidden_size = 단어 임베딩 사이즈) # !!!
        self.gru = nn.GRU(hidden_size, hidden_size) # nn.GRU(input_size = 단어 임베딩 사이즈, hidden_size = Sequence 사이즈, layer_size = GRU Stacking 개수) # !!!

    # 2] 순전파
    def forward(self, x, hidden):
        # [1] Word Embedding
        # print(x.size()) # torch.Size([1])
        output = self.embedding(x).view(1, 1, -1) # self.embedding(idx = 단어의 idx) # !!!
        # print(self.embedding(x)) # torch.size([hidden_size])
        # print(output.size()) # torch.Size([1, 1, hidden_size])

        # [2] RNN 통과
        output, hidden = self.gru(output, hidden) # self.gru(input_size = 단어 임베딩 사이즈, hidden_size = 단어 임베딩 사이즈) # !!!
        # print(output.size()) # torch.Size([1, 1, hidden_size])
        # print(hidden.size()) # torch.Size([1, 1, hidden_size])
        return output, hidden
    
enc_hidden_size = 100
encoder = Encoder(input_size=load_source_vocab.n_vocab, hidden_size=enc_hidden_size).to(device)

- 입출력 예시

In [78]:
hidden0 = torch.zeros([1, 1, encoder.hidden_size]).to(device) # !!!
# print(hidden0.size()) # torch.Size([1, 1, 100])
# print(hidden0)
"""
tensor([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0.]]], device='cuda:0')
"""

output1, hidden1 = encoder(a_sample_tensor[0], hidden0) 
# print(a_sample_tensor[0].size()) # 나는 -> torch.Size([1, 1, 100])
# print(output1.size()) # torch.Size([1, 1, 100])
# print(hidden1.size()) # torch.Size([1, 1, 100])
output2, hidden2 = encoder(a_sample_tensor[1], hidden1)
output3, hidden3 = encoder(a_sample_tensor[2], hidden2)

In [79]:
hidden0

tensor([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0.]]], device='cuda:0')

In [80]:
hidden1

tensor([[[ 1.9564e-01,  4.1397e-01,  3.9918e-01, -7.8937e-03,  8.2694e-02,
          -8.8768e-02,  3.1606e-01, -4.0960e-01, -3.5495e-01, -1.5009e-01,
          -2.8005e-01, -1.3460e-01, -4.3566e-01, -2.5684e-02, -3.7400e-02,
           3.5373e-01,  9.3664e-02, -3.0394e-01, -9.4407e-02, -1.6605e-01,
          -5.3931e-01,  8.4364e-02,  2.8361e-02, -1.2829e-01,  2.5686e-01,
           2.8616e-01, -1.2377e-01, -1.1872e-01, -8.6845e-02, -2.7486e-01,
          -1.8411e-01,  1.3232e-01,  3.9957e-01,  2.6912e-01,  6.0866e-01,
          -1.1086e-01, -4.1042e-01,  4.6750e-02, -1.9555e-01,  4.8074e-02,
           2.8557e-01, -2.5585e-02, -1.6374e-01, -3.9913e-02,  7.1142e-02,
           9.6246e-02,  1.5816e-01,  6.2053e-02,  1.3343e-01, -3.7224e-02,
           2.6560e-01, -2.0995e-01,  1.9137e-01, -2.2487e-01,  2.1246e-01,
          -9.8638e-02,  2.7650e-01,  2.0018e-01,  1.1567e-01, -4.0895e-01,
           2.7517e-01,  1.8715e-01,  1.8011e-01,  1.3214e-01, -6.3058e-01,
           6.3613e-02, -3

In [81]:
hidden2

tensor([[[-0.0965, -0.0308,  0.0864, -0.1396,  0.0600,  0.2152,  0.3431,
          -0.2439, -0.1648, -0.2527, -0.0446,  0.2394, -0.0091,  0.0229,
           0.1064,  0.3918,  0.2766, -0.4453,  0.1440, -0.2286, -0.4266,
          -0.0843, -0.0123, -0.3415,  0.2635,  0.0619, -0.1686,  0.2185,
          -0.2005, -0.1257, -0.3966, -0.2532, -0.1155, -0.0301,  0.4963,
          -0.2744, -0.2152, -0.1163, -0.3713, -0.1630,  0.4571,  0.2375,
          -0.3734, -0.1840, -0.2504, -0.0416, -0.0294,  0.4091, -0.0306,
           0.0174,  0.2199, -0.6136,  0.2615, -0.1821,  0.3236,  0.2608,
          -0.0207, -0.0621, -0.0164, -0.1053,  0.1960, -0.1680, -0.0231,
          -0.1897, -0.2216,  0.3936, -0.0917,  0.2908, -0.4966,  0.1360,
          -0.2142, -0.1636,  0.2003, -0.2887,  0.2708, -0.3849,  0.1877,
          -0.3600, -0.0837,  0.0076,  0.3150, -0.3315, -0.2911,  0.0208,
           0.0105,  0.3125,  0.3048, -0.3148, -0.2389,  0.2197, -0.2230,
          -0.0720,  0.1353,  0.3870,  0.2428,  0.28

In [82]:
hidden3

tensor([[[-0.1887,  0.6137, -0.3572, -0.1522,  0.3851,  0.0577,  0.5011,
          -0.6522,  0.2979, -0.0890, -0.2085,  0.4486,  0.0779,  0.1983,
          -0.3706,  0.0965,  0.1474, -0.2565,  0.0176, -0.2055, -0.5902,
           0.2861,  0.3917,  0.1829,  0.2988,  0.4491, -0.2608,  0.2959,
          -0.2795, -0.1065, -0.2670, -0.2717, -0.0340, -0.0072,  0.5189,
           0.4126, -0.1041, -0.6828, -0.2622,  0.1632,  0.0950,  0.6033,
          -0.1021,  0.2117, -0.3145, -0.2133, -0.2828,  0.1924,  0.0918,
          -0.1768,  0.4126, -0.4619,  0.2799, -0.0030, -0.3474,  0.2274,
          -0.4291, -0.1159,  0.2008,  0.2927, -0.3899, -0.2598, -0.0440,
           0.0675, -0.4606,  0.5991,  0.0084,  0.3445, -0.5692,  0.4948,
          -0.1518, -0.0876,  0.2156, -0.4396,  0.3661, -0.4074,  0.0988,
          -0.2172,  0.0336,  0.2232,  0.1701, -0.4544, -0.2279,  0.3393,
          -0.3165,  0.3913,  0.1338, -0.3276, -0.4367,  0.5603, -0.0868,
          -0.0102,  0.2036,  0.5282,  0.3084,  0.27

(2) Decoder 함수 정의

In [83]:
gru = nn.GRU(input_size=10, hidden_size=32, num_layers=1, batch_first=True, bidirectional=False)

gru # GRU(10, 32, batch_first=True)

GRU(10, 32, batch_first=True)

In [84]:
inputs = torch.zeros(1, 35, 10) # !!

print(inputs.shape) # torch.Size([1, 35, 10])
# [batch_size, sequence_length, number of features]

torch.Size([1, 35, 10])


In [85]:
outputs, hidden_state = gru(inputs) # !!

print(outputs.shape) # torch.Size([1, 35, 32]) # !!!
# [batch_size, sequence_length, hidden_size(32) x bidirectional(1)]
print(hidden_state.shape) # torch.Size([1, 1, 32]) # !!!
# [bidirectional(1) x number of layers(1), batch_size, hidden_size(32)]

torch.Size([1, 35, 32])
torch.Size([1, 1, 32])


In [86]:
gru = nn.GRU(input_size=10, hidden_size=32, num_layers=2, batch_first=True, bidirectional=False)

gru # GRU(10, 32, num_layers=2, batch_first=True)

GRU(10, 32, num_layers=2, batch_first=True)

In [88]:
inputs = torch.zeros(1, 35, 10)

print(inputs.shape) # torch.Size([1, 35, 10])
# [batch_size, sequence_length, number of features]

torch.Size([1, 35, 10])


In [89]:
outputs, hidden_state = gru(inputs)

print(outputs.shape) # torch.Size([1, 35, 32]) # !!!
# [batch_size, sequence_length, hidden_size(32) x bidirectional(1)]
print(hidden_state.shape) # torch.Size([2, 1, 32]) # !!!
# [Bidirectional(1) x number of layers(1), batch_size, hidden_size(32)]

torch.Size([1, 35, 32])
torch.Size([2, 1, 32])


In [90]:
# declare simple decoder
class Decoder(nn.Module):
    # 1] 모델 생성
    def __init__(self, hidden_size, output_size):
        super(Decoder, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(output_size, hidden_size) # nn.Embedding(input_size = Vocab 사이즈, hidden_size = 단어 임베딩 사이즈) # !!!
        self.gru = nn.GRU(hidden_size, hidden_size) # nn.GRU(input_size = 단어 임베딩 사이즈, hidden_size = Sequence 사이즈, layer_size = GRU Stacking 개수) # !!!
        self.out = nn.Linear(hidden_size, output_size) # !!!
        self.softmax = nn.LogSoftmax(dim=1) # !!!

    # 2] 순전파
    def forward(self, x, hidden):
        # print(x.size()) # torch.Size([1, 1])
        # print(hidden.size()) # torch.Size([1, 1, 100])

        # [1] Word Embedding
        output = self.embedding(x).view(1, 1, -1) # self.embedding(idx = 단어의 idx) # !!!
        # print(self.embedding(x)) # torch.size([hidden_size])
        # print(output.size()) # torch.Size([1, 1, 100])

        # [2] RNN 통과
        output, hidden = self.gru(output, hidden) # self.gru(input_size = 단어 임베딩 사이즈, hidden_size = Sequence 사이즈) # !!!
        # print(output.size()) # torch.Size([1, 1, 100])
        # print(hidden.size()) # torch.Size([1, 1, 100])

        # [3] Linear + Softmax
        output = self.softmax(self.out(output[0])) # !!!
        # print(output.size()) # torch.Size([1, 10008])
        # print(output[0].size()) # torch.Size([10008])

        return output, hidden

dec_hidden_size = enc_hidden_size
decoder = Decoder(hidden_size=dec_hidden_size, output_size=load_target_vocab.n_vocab).to(device)

- 입출력 예시

In [91]:
decoder_input = torch.Tensor([[SOS_token]]).long().to(device) # !!!
decoder_hidden = hidden3 # !!!
decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden) # !!!

In [92]:
decoder_hidden

tensor([[[-0.0311,  0.7133, -0.2925,  0.1929, -0.1929, -0.3186,  0.2190,
          -0.3818,  0.1504, -0.1134, -0.0624,  0.2981,  0.2722, -0.2421,
           0.0489,  0.0896,  0.1685,  0.1493,  0.0477, -0.0198, -0.5141,
           0.1653,  0.4175,  0.0664,  0.4062,  0.0894, -0.2497,  0.2148,
           0.0502,  0.0650,  0.1665,  0.0763, -0.3775, -0.0275,  0.1489,
          -0.1742, -0.2347, -0.3140, -0.4149,  0.2763,  0.1850,  0.0200,
          -0.2085,  0.0439,  0.0493, -0.3948, -0.1398,  0.4380, -0.2635,
          -0.2147,  0.2785, -0.4293,  0.4078, -0.0194, -0.2425,  0.4045,
           0.1561, -0.3089,  0.2467,  0.4940, -0.4679, -0.2793,  0.0441,
          -0.2765,  0.1680,  0.6161,  0.4891,  0.1722, -0.6098,  0.4225,
           0.3628, -0.2975, -0.2624, -0.3511,  0.5564, -0.3980, -0.2140,
           0.0885,  0.1283,  0.2380,  0.0849, -0.2218, -0.1215,  0.1531,
          -0.2012,  0.1178,  0.3234, -0.2681,  0.2838,  0.3409, -0.3967,
           0.0584, -0.2173,  0.1694,  0.3631, -0.13

##### 5-2) Train 함수 정의

In [93]:
# training seq2seq
def train(pairs, source_vocab, target_vocab, encoder, decoder, encoder_optimizer, decoder_optimizer, n_iter, print_every=1000, learning_rate=0.01):
    # (1) 모델 학습 선언
    encoder.train() # !!!
    decoder.train() # !!!
    # (2) 변수 선언
    loss_total = 0
    # (3) Cost 선언
    criterion = nn.NLLLoss() # !!!

    # (4) 전체 train 데이터 불러오기
    train_batch = [random.choice(pairs) for _ in range(n_iter)] # !!!
    # (5) 전체 train 데이터 변환
    train_sources = [str_to_indexes_tensor(source_vocab, pair[0]) for pair in train_batch] # !!!
    # print(f'train_sources : {train_sources}', flush=True)
    train_targets = [str_to_indexes_tensor(target_vocab, pair[1]) for pair in train_batch] # !!!
    # print(f'train_targets : {train_targets}', flush=True)

    # (6) Epoch 순회
    for i in range(1, n_iter + 1):
        # 1] Loss 초기화
        loss = 0
        # 2] 초기 hidden값 선언
        encoder_hidden = torch.zeros([1, 1, encoder.hidden_size]).to(device) # !!!
        # 3] train 데이터 하나씩 불러오기
        train_source = train_sources[i - 1] # !!!
        train_target = train_targets[i - 1] # !!!
        # print(f'train_source : {train_source}')
        # print(f'train_target : {train_target}')
        # 4] train 데이터 하나의 0차원 Size
        source_length = train_source.size(0) # !!!
        target_length = train_target.size(0) # !!!

        # 5] Encoder Hidden State 업데이트
        for train_source_idx in range(source_length): # !!!
            _, encoder_hidden = encoder(train_source[train_source_idx], encoder_hidden) # hidden state만 계속해서 바뀜 # !!!

        # 6] Decoder Input + Hidden 선언
        decoder_input = torch.Tensor([[SOS_token]]).long().to(device) # !!!
        decoder_hidden = encoder_hidden # connect encoder output to decoder input # !!!
        # 7] Decoder 문장 생성 + Loss 계산
        for train_target_idx in range(target_length): # !!!
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden) # !!!
            loss += criterion(decoder_output, train_target[train_target_idx]) # !!!
            decoder_input = train_target[train_target_idx]  # teacher forcing # !!!

        # 8] 모델 역전파
        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()

        loss.backward() # Decoder만 Loss 존재 !!!

        encoder_optimizer.step()
        decoder_optimizer.step()

        # 9] 전체 Loss 계산
        loss_iter = loss.item() / target_length # !!!
        loss_total += loss_iter

        # 10] Loss 출력
        if i % print_every == 0:
            loss_avg = loss_total / print_every # !
            loss_total = 0
            print("[{} - {}%] loss = {:05.4f}".format(i, i / n_iter * 100, loss_avg))

##### 5-3) Evaluate 함수 정의

In [94]:
# insert given sentence to check the training
def evaluate(pairs, source_vocab, target_vocab, encoder, decoder, target_max_length, print_every=5):
    # (1) 모델 평가 선언
    encoder.eval() # !!!
    decoder.eval() # !!!

    # (2) 한 Pair씩 Eval
    for idx, pair in enumerate(pairs):
        # 1] Eval 데이터 하나씩 불러오기 -> 변환
        eval_source = str_to_indexes_tensor(source_vocab, pair[0]) # !!!
        # 2] Eval 데이터 하나의 0차원 Size
        source_length = eval_source.size()[0]

        # 3] Encoder Hidden 선언
        encoder_hidden = torch.zeros([1, 1, encoder.hidden_size]).to(device)
        # 4] Encoder Hidden State 추출
        for eval_source_idx in range(source_length):
            _, encoder_hidden = encoder(eval_source[eval_source_idx], encoder_hidden)

        # 5] Decoder Input + Hidden 선언
        #decoder_input = torch.Tensor([[SOS_token]], device=device).long()
        decoder_input = torch.Tensor([[SOS_token]]).long().to(device)
        decoder_hidden = encoder_hidden
        predicted_words = []
        # 6] Decoder 문장 생성
        for di in range(target_max_length): # !!!
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            _, top_index = decoder_output.data.topk(1) # !!!
            if top_index.item() == EOS_token: # !!!
                predicted_words.append("<EOS>")
                break
            else:
                predicted_words.append(target_vocab.index2vocab[top_index.item()])

            decoder_input = top_index.squeeze().detach() # !!!

        predicted_sentence = " ".join(predicted_words)
        # 7] 예측 결과 출력
        if idx % print_every==0:
          print("입력 : ", pair[0]) 
          print("실제 : ", pair[1])
          print("예측 : ", predicted_sentence)
          print("")

##### 7) 모델 정의

In [95]:
enc_hidden_size = 100
dec_hidden_size = enc_hidden_size
encoder = Encoder(input_size=load_source_vocab.n_vocab, hidden_size=enc_hidden_size).to(device)
decoder = Decoder(hidden_size=dec_hidden_size, output_size=load_target_vocab.n_vocab).to(device)

##### 8) Optimizer 정의

In [96]:
learning_rate = 0.01
encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)

##### 9) 모델 학습

In [97]:
train(load_pairs, load_source_vocab, load_target_vocab, encoder, decoder, encoder_optimizer, decoder_optimizer, n_iter=len(load_pairs)*5, print_every=500)

[500 - 0.8458090163241141%] loss = 7.5607
[1000 - 1.6916180326482282%] loss = 6.9406
[1500 - 2.537427048972342%] loss = 6.7895
[2000 - 3.3832360652964564%] loss = 6.5898
[2500 - 4.22904508162057%] loss = 6.5948
[3000 - 5.074854097944684%] loss = 6.4587
[3500 - 5.920663114268798%] loss = 6.4837
[4000 - 6.766472130592913%] loss = 6.4970
[4500 - 7.6122811469170255%] loss = 6.3769
[5000 - 8.45809016324114%] loss = 6.3805
[5500 - 9.303899179565255%] loss = 6.3676
[6000 - 10.149708195889367%] loss = 6.2131
[6500 - 10.995517212213482%] loss = 6.2486
[7000 - 11.841326228537596%] loss = 6.1908
[7500 - 12.687135244861711%] loss = 6.1481
[8000 - 13.532944261185825%] loss = 6.2347
[8500 - 14.378753277509936%] loss = 6.1236
[9000 - 15.224562293834051%] loss = 6.1070
[9500 - 16.070371310158166%] loss = 6.0914
[10000 - 16.91618032648228%] loss = 6.0708
[10500 - 17.761989342806395%] loss = 6.0120
[11000 - 18.60779835913051%] loss = 6.0314
[11500 - 19.453607375454624%] loss = 5.9766
[12000 - 20.2994163

##### 10) 모델 예측

In [98]:
evaluate(load_pairs, load_source_vocab, load_target_vocab, encoder, decoder, TARGET_MAX_LENGTH, print_every=5000)

입력 :  12시 땡!
실제 :  하루가 또 가네요.
예측 :  하나씩 <EOS>

입력 :  학원폭력 짜증나
실제 :  학교 폭력은 범죄에요.
예측 :  다시 연락해보세요. <EOS>

입력 :  사랑한다고 말해주면 뭐가 덧나나
실제 :  사랑한다고 표현해달라고 말해보세요.
예측 :  다른 반 친구한테 <EOS>



In [99]:
a_test_sample = [['무슨 개소리를 이렇게 장황하게해', '?']]

In [100]:
evaluate(a_test_sample, load_source_vocab, load_target_vocab, encoder, decoder, TARGET_MAX_LENGTH)

입력 :  무슨 개소리를 이렇게 장황하게해
실제 :  ?
예측 :  같이 가보세요. <EOS>



## 실습0: seq2seq 모델의 모델 부분을 구현해 보세요!

## 실습1: Encoder 부분을 조금 더 효율적으로 구현할 수 없을까? (병렬처리....)

## 실습2: Word Embedding, Language Model 이 두개가 제일 중요하다며? 이 두개는 어떻게씀?? 
 - 사전학습된 Word Embedding을 쓰면 성능 올라 단어임베딩을 넣으면 성능이 올라가겠지? 대신 험난한 여정이...
 - Language Model을 어떻게 사전학습해서 이 모델에 사용할 수 있을까?? (다음시간에)
 